In [41]:
import numpy as np
import tensorflow as tf

In [42]:
with open('xids.npy', 'rb') as fp:
    Xids = np.load(fp)
    
with open('xmask.npy', 'rb') as fp:
    Xmask = np.load(fp)
    
with open('labels.npy', 'rb') as fp:
    labels = np.load(fp)

In [43]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

In [44]:
def map_func(input_ids, mask, label):
    return{'input_ids': input_ids, 'attention_mask': mask}, label

In [45]:
BATCH_SIZE= 32
dataset = dataset.shuffle(10000).batch(BATCH_SIZE)

In [46]:
SPLIT = 0.8
DS_LEN = len(list(dataset))

train = dataset.take(round(DS_LEN*SPLIT))
val = dataset.skip(round(DS_LEN*SPLIT))

In [47]:
from transformers import TFAutoModel

In [48]:
bert = TFAutoModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [53]:
input_ids = tf.keras.Input(shape=(30,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(30 ,), name='attention_mask', dtype='int32')

embeddings = bert(input_ids, attention_mask=mask)[0]

X = tf.keras.layers.Conv1D(filters=50, kernel_size=2, padding='valid', activation='relu')(embeddings)
X = tf.keras.layers.Conv1D(filters=50, kernel_size=2, padding='valid', activation='relu')(X)
X = tf.keras.layers.Conv1D(filters=50, kernel_size=2, padding='valid', activation='relu')(X)
X = tf.keras.layers.GlobalMaxPool1D()(X)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation='relu')(X)
X = tf.keras.layers.Dropout(0.1)(X)
X = tf.keras.layers.Dense(32, activation='relu')(X)
y = tf.keras.layers.Dense(3, activation='softmax', name='outputs')(X)


model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

model.layers[2].trainable = False


In [54]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 30)]         0           []                               
                                                                                                  
 tf_bert_model_2 (TFBertModel)  multiple             108310272   ['input_ids[0][0]',              
                                                                  'attention_mask[0][0]']         
                                                                                                  
 conv1d (Conv1D)                (None, 29, 50)       76850       ['tf_bert_model_2[1][0]']  

In [55]:
optimizer = tf.keras.optimizers.Adam(0.01)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [56]:
history = model.fit(
    train,
    validation_data=val,
    epochs=20
)

Epoch 1/20


ValueError: in user code:

    File "/Users/nikhilsaireddychoppa/opt/anaconda3/envs/mlenv/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/nikhilsaireddychoppa/opt/anaconda3/envs/mlenv/lib/python3.10/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/nikhilsaireddychoppa/opt/anaconda3/envs/mlenv/lib/python3.10/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/nikhilsaireddychoppa/opt/anaconda3/envs/mlenv/lib/python3.10/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/Users/nikhilsaireddychoppa/opt/anaconda3/envs/mlenv/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(fi